## Preorder Travesal and Postorder traversal


<strong>Running time analysis</strong>


Both ptreorder and psotorder traversal algorithms are efficient ways to access all the position of a tree
At each position p, the non recursive part of the traversal algorithm requires time(Ocp+1), where cp is the number of children of p under the assumption that the "visit" itself takes O(1)
Overall running time for the traversal of tree is O(n)



In [6]:
class Tree:
    """An abstract base class representing a tree structure"""

    # --------nested Position class-----
    class Position:
        """An abstraction representing a location of a single element"""

        def element(self):
            """Return the element stored at this position"""
            raise NotImplementedError('must be implemented by a subclass')

        def __eq__(self, other):
            """return true if the position represents the same location"""
            raise NotImplementedError('must be implemented by a subclass')

        def __ne__(self, other):
            """Return true if the other does not represent the same location"""
            return not (self == other)

    # -------abstract methods that concrete subclass must support-----
    def root(self):
        """Return Position representing the trees root"""
        raise NotImplementedError('must be implemented by a subclass')

    def parent(self, p):
        """Return the postion representing P's parent"""
        raise NotImplementedError('must be implemented by a subclass')

    def num_children(self, p):
        """Return the number of the children that position p has"""
        raise NotImplementedError('must be implemented by a subclass')

    def children(self, p):
        """Generate an iteration og Postions representing p's children"""
        raise NotImplementedError('must be implemented by a subclass')

    def __len__(self):
        """Return the total number of elements in the tree"""
        raise NotImplementedError('must be implemented by a subclass')

    # ---------concrete methods implemented in this class-----
    def is_root(self, p):
        """Return True if the element p is the root"""
        return self.root() == p

    def is_leaf(self, p):
        """Return true if the element p does not have any children"""
        return self.num_children(p) == 0

    def is_empty(self):
        """Return true if the tree is empty"""
        return len(self) == 0

    def depth(self, p):
        """Return the number of levels separating Position p from the root"""
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))

    def _height2(self, p):
        """Return the height of the tree at posititon p"""
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._height2(c) for c in self.children(p))

    def height(self, p=None):
        """return the height of the tree at position p

        If there is no p then return the height of the entire tree"""
        if p is None:
            p = self.root()
        return self._height2(p)

    def __iter__(self):
        """Generate an iteration of the trees elements """
        for p in self.positions():
            yield p.element()
            
    def preorder(self):
        """Generate a preorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p
                
    def _subtree_preorder(self, p):
        """Generate a preorder iteration of positions in subtree rooted at p"""
        yield p                                             #visit p before its subtree
        for c in self.children(p):                             #for each child
            for other in self._subtree_preorder(c):         #do preorder of c's subtree
                yield other                                 #yielding each to our caller
    
    
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.preorder()          #return the entire preorder iteration
    
class BinaryTree(Tree):
    """an abstract base class representing a binary tree structure"""

    # -----additional abstract methods-------
    def left(self, p):
        """Return the Position representing p's left child
        return None if p does not have a left child"""
        raise NotImplementedError('must be implemented by a subclass')

    def right(self, p):
        """Return the Position representing p's right child
        return None if p does not have a right child"""
        raise NotImplementedError('must be implemented by a subclass')

    # -----concrete methods implemented in this class----
    def sibling(self, p):
        """Return a Position representing p's sibling
        (or none if no sibling)"""
        parent = self.parent(p)
        if parent is None:  # p must be the root
            return None  # root has no sibling
        else:
            if p == self.left(parent):  # if p is the left child of the parent
                return self.right(parent)  # return the right child of the parent (possibly None)
            else:
                return self.left(parent)

    def children(self, p):
        """Generate an iteration of Positions representing P's children"""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

class LinkedBinaryTree(BinaryTree):

    """Linked representation of a binary tree"""

    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'
        def __init__(self, element, parent = None, left = None, right = None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right


    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element"""

        def __init__(self, container, node):
            """Constructor should not be invoked by the user"""
            self._container = container
            self._node = node


        def element(self):
            # return the element stored at this position
            return self._node._element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node


    def _validate(self, p):
        """Return the associated node if position is valid"""
        if not isinstance(p, self.Position):
            raise TypeError('p must be at valid position')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node


    def _make_position(self, node):
        """Return Position instance for given node"""
        return self.Position(self, node) if node is not None else None


    # --------------BINARY TREE CONSTRUCTOR ---------

    def __init__(self):
        """Create an initially empty binary tree"""
        self._root = None
        self._size = 0

    # --------PUBLIC ACCESSORS---------
    def __len__(self):
        return self._size

    def root(self):
        """Return the root position of the tree"""
        return self._make_position(self._root)

    def parent(self, p):
        """Return the position of the p's parent (or None if p is root)"""
        node = self._validate(p)
        return self._make_position(node._parent)

    def left(self, p):
        """Return the position of the p's left child(or None if no left child)"""
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        """Return the position of the p's right child(or None if no right child)"""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p"""
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count


    #----NON PUBLIC UPDATE METHODS-----
    def _add_root(self, e):
        """Place element at the root of an empty tree and return new Position
        raise ValueError if tree is non empty"""
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        """Create a new left child for a postion P storing element e.
        return the position of the new node
        raise a ValueError if the node already has a left child"""
        node = self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)

    def _add_right(self, p, e):
        """Create a new right child for a postion P storing element e.
        return the position of the new node
        raise a ValueError if the node already has a right child"""
        node = self._validate(p)
        if node._right is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)

    def replace(self, p, e):
        """Replace the element at position p with e and return old element"""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        """Delete the node at Position p and replace it with its child, if any
        Return the element that had been stored at Position p
        raise ValueError if position p is invalid or p has two children"""
        node = self._validate(p)
        if self.children(p) == 2: raise ValueError('p has two children')
        child = node._left if node._left else node._right           #might be none
        if child is not None:
            child._parent = node._parent        #child grandparent becomes the parent
        if node is self._root:
            self._root = child          #child becomes the root
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child

        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):
        """Attach t1 and t2 as a subtree at external p"""
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('p must be leaf')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('trees must be of same type')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2.size = 0


In [ ]:
LinkedBinaryTree.__init__(self)

TypeError: __init__() missing 1 required positional argument: 'self'